# Turtle, Command Line Arguments, Bash Script Wrappers

Sources: 
- [Visualizing Recursion](https://runestone.academy/runestone/books/published/pythonds/Recursion/pythondsintro-VisualizingRecursion.html)
- see supplementary material

# Turtle module

`Turtle` is an unsophisticated drawing module for python. In this paradigm, rather than using an absolute coordinate system (eg. X,Y), the _turtle_ uses a relative coordinate system centered on it. Therefore it accepts commands such as `forward(length)`, `backward(length)`, `righ(degrees)`, `left(degrees)`, etc. It has a _pen_ on its belly and can `penup()` to stop drawing and `pendown()` to resume. `color(pen_color)` can change the color of the pen.

In [3]:
#!pip3 install ColabTurtle # Turtle for colab
! pip install --user ColabTurtle

  Using cached ColabTurtle-2.0.0.tar.gz (3.3 kB)
  Created wheel for ColabTurtle: filename=ColabTurtle-2.0.0-py3-none-any.whl size=4028 sha256=0cf6c49d9a71c41b74699d01e6eae35858909cb768be9141fc480085f43ee8d7
  Stored in directory: /Users/valazeinali/Library/Caches/pip/wheels/10/a4/55/1b309631a521da287927e20fbc43f8780328916a78c004ef32
Successfully built ColabTurtle


There are multiple ways to import the module and its `Turtle`. This is a good time to use an _alias_, because of the large number of `Turtle` commands.

In [4]:
from ColabTurtle import Turtle as T   # import the turtle

Then the _frame_ has to be initialized and the turtle dropped in the middle

In [5]:
T.initializeTurtle()

The arrow shows where the head of the turtle points.

In [6]:
T.right(90)
T.forward(50) # moves froward 100 units
T.right(90) # turn 90 degrees right
T.forward(50)
T.right(90)
T.forward(50)
T.right(90)
T.forward(50)

Or better, create a function `square`

In [14]:
def square(side=100):
  T.right(90)
  T.forward(side) # moves froward 100 units
  T.right(90) # turn 90 degrees right
  T.forward(side)
  T.right(90)
  T.forward(side)
  T.right(90)
  T.forward(side)

T.initializeTurtle()
square()

Modify, and repeat... a new square after turning the turtle 10 degrees.

In [19]:
T.initializeTurtle()
T.speed(10)
angle = 360
while angle > 0:
  square()
  T.left(10)
  angle -=10

This is when recursion becomes interesting. You can use recursion to draw ever smaller squares...

In [24]:
def corner_square(side=200):
  square(side)
  if side > 5: # stopping condition
    corner_square(int(side*.99))

T.initializeTurtle()
T.penup()
T.forward(200)
T.right(90)
T.backward(200)
T.left(90)
T.pendown()
T.speed(10)
corner_square(400)

An not just squares, but trees... recursive trees, where each tree is made up of smaller trees until the length of the branch is below 5 units.

In [10]:
from ColabTurtle import Turtle as T

def tree(branchLen):
    if branchLen > 5:         # stopping condition
        T.forward(branchLen)  # draw trunk
        T.right(20)           # turn right at the end of trunk
        tree(branchLen-10)    # draw a smaller "right tree" at the end of the trunk
        T.left(40)            # turn to draw "left sub-tree"
        tree(branchLen-10)    # draw a smaller "left tree"
        T.right(20)           # undo angle turns
        T.backward(branchLen) # go back to tree root

def main():                   
    T.initializeTurtle()      # clear frame
    T.speed(10)               # max speed
    T.penup()                 # position the tree root to the bottom of the frame
    T.backward(200)
    T.pendown()
    T.color("green")          # geen pen
    tree(85)                 # draw the first branch (ie. trunk) 
    
main()


# Command line arguments

Most programs require information provided by the user _before_ execution. In fact, well designed software will _collect_ all the information upfront and then not interact with the user until it is done processing or an error occures.

Rather than asking the user for `input` one-by-one, Python, like most programming languages offers the option to pass those arguments on the _command line_ at the time the Python script is called. Command line arguments are used to interact with a "black box" code and are especially useful when automating their run over multiple values.

This is most useful in a terminal setting, as Colab and other Notebook environments do not easily support this feature.

Let's use an example of a function that is general enough that it will require some input from the user OR _command line arguments_.

The code below implements a calculator.

In [11]:
def add(a,b):
  return a+b

def substract(a,b):
  return a-b

def multiply(a,b):
  return a*b

def divide(a,b):
  try: 
    return a/b
  except ZeroDivisionError:
    print("Zero Division")
    return None

def perform_operation(operation, a, b):
  '''
  Performs the specified operation using the 2 arguments and returns the result, or None if an error occures
  '''
  operations = {'+': add, '-': substract, '*': multiply, '/': divide}

  try: # attempts to convert a and b into floats
    a = float(a)
    b = float(b)
  except (ValueError,TypeError) as e:
    print("At least one argument cannot be converted to a float: ",str(e))
    return None

  op = operations.get(operation) # gets the operation function from dictionary
  if op is None:
    print(f'Operation "{operation}" is invalid, choose between +,-,*,/')
    return None

  return op(a,b)

input_string = input('Operation to parse (eg. "10 * 20", incl. spaces): ')
splits = input_string.split(' ')

print(f'{" ".join(splits)} = {perform_operation(splits[1],splits[0],splits[2])}')


Operation to parse (eg. "10 * 20", incl. spaces): 10 * 20 
10 * 20  = 200.0


Rather than (or in addition to) passing the arguments using the `input` function, lets read the input from the command line arguments.

The code below uses the `sys` module's `argv` attribute to _read_ arguments passed on the command line.
```
'''
Command line argument script: cmd_line.py
'''
import sys

print(type(sys.argv))
print(sys.argv)

for arg in sys.argv:
  print(arg)
```
Therefore, when called in the terminal:
```
  $ python cmd_line.py a bc 123
```
will print
```
  <class 'list'>
  ['cmd_line.py','a', 'bc', '123']
  cmd_line.py
  a
  bc
  123
```
The arguments passed on the command line after the name of the script file are available in the code as a list in `sys.argv`.

Let's add the option to pass the same argument on the command line rather than using `input`.

In [17]:
import sys    # importing the sys module

def add(a,b):
  return a+b

def substract(a,b):
  return a-b

def multiply(a,b):
  return a*b

def divide(a,b):
  try: 
    return a/b
  except ZeroDivisionError:
    print("Zero Division")
    return None

def perform_operation(operation, a, b):
  '''
  Performs the specified operation using the 2 arguments and returns the result, or None if an error occures
  '''
  operations = {'+': add, '-': substract, '*': multiply, '/': divide}

  try: # attempts to convert a and b into floats
    a = float(a)
    b = float(b)
  except (ValueError,TypeError) as e:
    print("At least one argument cannot be converted to a float: ",str(e))
    return None

  op = operations.get(operation) # gets the operation function from dictionary
  if op is None:
    print(f'Operation "{operation}" is invalid, choose between +,-,*,/')
    return None

  return op(a,b)

splits = sys.argv[1:]
if len(splits)!=3:
  input_string = input('Operation to parse (eg. "10 * 20", incl. spaces): ')
  splits = input_string.split(' ')

print(f'{" ".join(splits)} = {perform_operation(splits[1],splits[0],splits[2])}')


Operation to parse (eg. "10 * 20", incl. spaces): 2 + 9
2 + 9 = 11.0


I saved the script above in a `calculator.py` file in my DartFS home. When running this code on HPC:
```
(qbs) [DarabosC@polaris week10]$ python calculator.py 5 + 6
5 + 6 = 11.0
(qbs) [DarabosC@polaris week10]$ python calculator.py
Operation to parse (eg. "10 * 20", incl. spaces): 56 + 4    
56 + 4 = 60.0
```
Both work: the command line arguments and the input, if the command line arguments are missing.

More generally, scripts will not ask the user for input if command line arguments are missing, but rather, they will have:
- default values for all missing command line arguments and/or
- fail and display an error message and help 

Mostly, command line arguments are used to specify the expected behavior and the input data of a script. By using command line arguments, script can be used without understanding (even looking at) the source code.

# Bash Script Wrapper

Oftentimes, the source code is unavailable (in compiled languages) or impractical to edit unless absolutely necessary. It may, however, be useful to run the code on multiple input values (eg. all $a,b \in [0,100]$, all files in a folder, etc.)

In those cases, the most common thing to do is to write a wrapper around the code. This wrapper could be in Python or any other language, but most commonly in the terminal interpreter language (eg. Bash). In fact, our previous code is not "python wrapper" friendly and would require some adapting to work with the Python wrapper below.


In [ ]:
'''
Python wrapper for the calculator, assugiming that the calculator comes in 
a calculator.py script file
'''
#import calculator.py <<<<< uncomment when running on HPC

'''
Running all possible operation on all combitanions of a,b in [0,100] integers
'''

for op in ['+','-','*','/']:
  for a in range(101):
    for b in range(101):
      print(f'{a}{op}{b} = {perform_operation(op, a, b)}')

An easier way that does not require any modification of the code is to use a bash script. It also alines with the idea that by running the Python code multiple times, it might take a long time and/or a lot of resources, and thus should be run on HPC.

The bash script below uses the Python script as-is and passes all combinations of operations and values as command line arguments.

`calc_wrapper.sh`

```
#!/bin/bash

for op in '+' '-' '*' '/' # loop over all operations
do
  for a in `seq 0 10` # loop over a sequence of numbers
  do
    for b in `seq 0 10` # loop over a sequence of numbers
    do
      python calculator.py $a $op $b  # execute the python code using command line arguments and bash parameters
    done
  done
done
```

Running the code in a terminal window:
```
  $ bash calc_wrapper.sh
```
will run each the Python script once for each operation, and values of `a` and `b`.

All scripts [here](https://drive.google.com/drive/folders/1UGVsLb_tkOnQDAZd3U4ouBV55oAxpO9X?usp=sharing).

In [ ]:
def draw(side):
    T.forward(side)
    T.right(1)
    draw(side)
    

T.initializeTurtle()
T.speed(10)
draw(1)

RecursionError: maximum recursion depth exceeded while calling a Python object